# Walmart sales forecasting using RNN (LSTM )
1. data preprocessing.

Some things to con
2. visualize the data to get a general idea



# References:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import os

# 1. data preprocessing 

In [ ]:
filepath = 'AMZN_2006-01-01_to_2018-01-01.csv'
data = pd.read_csv(os.path.join('data', filepath))
data = data.sort_values('Date')
data.head()